In [100]:
import xlrd
import os
import numpy as np
import pandas as pd
hf = pd.DataFrame({})

workbook = xlrd.open_workbook("Closed End Fixed Income.xlsx")

names = workbook.sheet_names()

for x in range(0,workbook.nsheets):
    sheet = workbook.sheet_by_index(x)
    df = pd.DataFrame([sheet.row_values(x) for x in range(1, sheet.nrows)])
    
    df.columns = df.iloc[0]
    df = df[1:]
    name = names[x]
    if name not in ["HYG Index", "JNK Index"]:
        df.to_csv("funds_fixed_income/"+name.upper()+".csv")


In [101]:
    
hf = pd.DataFrame({})
my_dir = os.listdir("./funds_fixed_income")
my_dir.remove(".DS_Store")
for x in my_dir:
    df = pd.read_csv("./funds_fixed_income/"+x)
    name = x.split(".csv")[0]
    cf = df["ID.WEIGHTS"].to_frame(name)
    #.iloc[:,-1]
    cf.index = df["ID"].replace(np.nan, "N/A")
    cf = cf.dropna(axis=0)
    na = cf[cf.index=="N/A"].astype('float').sum(axis=0)
    na += cf.iloc[cf.index.str[:4]=="#N/A"].sum(axis=0)
    cf = cf[(cf.index != "N/A") & (cf.index.str[:4]!="#N/A")]
    cf.loc["N/A"] = na
    cf = cf.groupby(level=0).sum()
    hf = hf.merge(cf, how='outer', left_index=True, right_index=True)
    

hf = hf.fillna(0)

#hf.index = pd.Series(hf.index).replace(np.nan, "N/A")
#hf = hf.reindex(columns=["aod","adx","peo", "aio"])
hf = hf.astype('float')


In [102]:
tf = pd.read_csv("./jnk_index.csv")
sf = tf[["ID", "ID.WEIGHTS"]]

#.iloc[:,-1:]

sf.columns = ["Symbol","Weight"]
#https://www.slickcharts.com/sp500
hf = hf.div(hf.sum(), axis=1)*100
#hf[hf.columns[hf.loc["N/A"]<=20]]


In [103]:
funds = hf.columns
df = pd.read_csv("Discount_fixed_income_HY.csv", index_col=0)

my_dir = os.listdir("./funds_fixed_income")
my_dir.remove(".DS_Store")
funds = list(map(lambda x: x.split(".")[0].upper(), my_dir))
df = df[df.index.isin(funds)]
df = df.iloc[:,-3:-1]
#print(list(filter(lambda x: x not in list(df.index), funds)))
df.columns = ["Discount", "52W Discount"]
df["Discount"] = list(map(lambda x: str(x).strip("%"), list(df["Discount"])))

df["52W Discount"] = [0 if x=="--" else str(x).strip('%') for x in list(df["52W Discount"])]
df = df.astype('float')
df["Discount_Weights"] = ((df["Discount"]+ (df["Discount"] - df["52W Discount"]))+100)/100
discount_df = df[['Discount_Weights']]
discount_df = df.reindex([x.upper() for x in hf.columns])


[]


In [75]:


def change_down(weights, find, no_change_nums):
    k = [x + (.01/(len(weights)-len(no_change_nums)-1)) for x in weights[:find]+weights[find+1:]]
    k.insert(find, weights[find]-.01)
    for x in no_change_nums:
        k[x] = weights[x]
    return k


def change_up(weights, find, no_change_nums):
    l = weights
    k = [x - (.01/(len(weights)-len(no_change_nums)-1)) for x in weights[:find]+weights[find+1:]]
    k.insert(find, weights[find]+.01)
    for x in no_change_nums:
        k[x] = weights[x]
  
    negs = abs(sum(list(filter(lambda x: max(x,0)==0, k))))
    zeros = list(filter(lambda x: max(k[x],0)==0, range(len(k))))
    for x in zeros:
        k[x] = 0
    zeros.append(find)
    zeros = zeros + no_change_nums
    
    not_zeros = list(filter(lambda x: x not in zeros, range(len(k))))
    for x in not_zeros:
        k[x] = k[x] - (negs/len(not_zeros))

    if negs != 0 and not_zeros == []:
        return False
   
    return k

def find(weights):
    global hf
    global sf
    df = hf.multiply(weights).sum(axis=1).to_frame("ours")
    df = df.merge(sf, how="outer", right_on='Symbol', left_index=True)
    df = df.set_index("Symbol")
    #df = df.drop(["Symbol"], axis=1)
    df = df.fillna(0)
    w = [max(x,0) for x in weights]
    val = sum(discount_df['Discount_Weights'] * w)
    [max(x,0) for x in weights]
   
    close_val = (abs(sum(filter(lambda x: x < 0, df["Weight"]-df["ours"])))+100)
    #close_val = sum([abs(2*x) if x < 0 else x for x in df["Weight"]-df["ours"]])
    #close_val = sum([abs(5*x) if x < 0 else x for x in df["Weight"]-df["ours"]])
    #close_val = sum([ 5*x for x in filter(lambda x: x>0, df["ours"]-df["Weight"])])
    return 2 * close_val * val

def best(type, weights, ideal, ideal_weights):
    if weights == False:
        return ideal_weights
    if list(filter(lambda x: x < 0, weights)) != list():
        return ideal_weights
    global no_change_nums
    global find_num
    num = find(weights)
    if num < ideal:
        if type == "up":
            if weights[find_num] + .01 > 0:
               
                return best("up",  change_up(weights, find_num, no_change_nums), num, weights)
            else:
                return weights
        if type == "down":
            
            if weights[find_num] - .01 > 0:
                return best("down", change_down(weights, find_num, no_change_nums), num, weights)
            else: 
                return weights
    else:
        return ideal_weights
    
    
    


def go_through(weights):
    global hf
    global sf
    global no_change_num
    global find_num
    num = find(weights)
    dw = change_down(weights, find_num, no_change_nums)
    uw = change_up(weights, find_num, no_change_nums)
    if uw == False and dw == False:
        return weights
    elif uw == False:
        return best("down", dw,num, weights)
    elif dw == False:
        return (best("up", uw,num, weights))
    up = find(uw)
    down = find(dw)
    if up < down and up < num:
        return (best("up", uw,num, weights))
        
    elif down < num:
        return best("down", dw,num, weights)
    else:
        return weights
    
        




In [108]:
final = pd.DataFrame(columns=["funds","weights"])


how_much_dis = [100, 0, -1, -2, -3,-4,-5,-6,-7,-8,-9,-10]
for x in how_much_dis:
    print(x)
    discount_df = discount_df[discount_df["Discount"]<=x]
    hf = hf[discount_df.index]
    t = []
    z = len(hf.columns)-1

    def path(y):
        global z
        if y == z:
            return "hey"
        global t
        for x in range(y,-1,-1):
            h = [x for x in range(y+1)]
            h.remove(x)
            t.append([x, h])
        path(y+1)
        if y == 0:
            return t


    def path2(y):
        if y == 0:
            return "hey"
        global t
        global z
        for x in range(y,z+1):
            h = [x for x in range(y,z+1)]
            h.remove(x)
            t.append([x, h])
        path(y-1)
        if y == z:
            return t


    path(0)
    path2(z)
    way = t


    weights = [1/len(hf.columns) for x in range(len(hf.columns))]
    n = 0
    for x in way:
        find_num = x[0]
        no_change_nums = x[1]
        weights = go_through(weights)

        n+=1


    def findy(weights):
        global hf
        global sf
        df = hf.multiply(weights).sum(axis=1).to_frame("ours")
        df = df.merge(sf, how="outer", right_on='Symbol', left_index=True)
        df = df.set_index("Symbol")
        #df = df.drop(["Symbol"], axis=1)
        df = df.fillna(0)
        val = sum(discount_df['Discount_Weights'] * weights)
        #close_val = sum(abs(df["ours"]-df["Weight"]))
        close_val = (abs(sum(filter(lambda x: x < 0, df["Weight"]-df["ours"])))+100)
        #close_val = sum([abs(2*x) if x < 0 else x for x in df["Weight"]-df["ours"]])
        print(close_val,val)

    findy(weights)

    weights = [round(x,5) for x in weights]

    #weights = list(map(lambda x: max(x,0),weights))
    #DF WITH JUST THE ONES WITH WEIGHTS
    #hf.loc["weights"] = weights
    H = list(filter(lambda x: x != False,[hf.columns[x] if weights[x]>0 else False for x in range(len(weights))]))
    hf2 = hf[H]

    weights_not_zero = list(filter(lambda x: x > 0,weights))

    df = hf2.multiply(weights_not_zero).sum(axis=1).to_frame("ours")
    df = df.merge(sf, how="outer", right_on='Symbol', left_index=True)
    df = df.set_index("Symbol")
    df = df.fillna(0)
    df = df.loc[~(df==0).all(axis=1)]
    tf = df[df['Weight']!=0]
    not_in = df[df['Weight']==0]["ours"].sum()
    tf = tf.rename(columns={"Weight":"Index"})
    tf = tf.sort_values(by=['ours'], ascending=False)
    i = pd.DataFrame([[not_in, 0]], ["notINs&p500"], columns=["ours","Index"])
    tf = tf.append(i)
    pd.set_option("display.max_rows", None, "display.max_columns", None)
    tf.head()


    #pd.set_option("display.max_rows", None, "display.max_columns", None)
    tf
    tf["more"] = tf["Index"] -tf["ours"]
    amount_more = abs(tf[tf["more"]<0]["more"]).sum()/(abs(tf[tf["more"]<0]["more"]).sum()+100)
    amount_sp500 = (100/(abs(tf[tf["more"]<0]["more"]).sum()+100))
    #amount_not_in_sp500 = tf.loc["notINs&p500"]["ours"]/(abs(tf[tf["more"]<0]["more"]).sum()+100)+tf.loc["notINs&p500"]["ours"]

    amount_sp500 + amount_more
    amount_buy = tf[tf["more"]>0]["more"].sum()/(abs(tf[tf["more"]<0]["more"]).sum()+100)

    print(amount_sp500)

    funds = hf.columns
    df = pd.read_csv("./Discount_fixed_income_HY.csv", index_col=0)
    my_dir = os.listdir("./funds_fixed_income")
    my_dir.remove(".DS_Store")
    funds = list(map(lambda x: x.split(".")[0].upper(), my_dir))
    df = df[df.index.isin(funds)]
    df = df.iloc[:,-3:-1]
    df.columns = ["Discount", "52W Discount"]

    df["Discount"] = list(map(lambda x: str(x).strip("%"), list(df["Discount"])))

    df["52W Discount"] = [0 if x=="--" else str(x).strip('%') for x in list(df["52W Discount"])]
    df = df.astype('float')
    df = df.reindex(H,axis=0)
    df["Weights"] = weights_not_zero
    df.head(100)
    print(sum(weights))
    print((df["Discount"]*df["Weights"]).sum()*amount_sp500)
    final.loc[len(df)] = ["h","i"]



100
160.09534485802124 0.9412749240263545
0.6331794322151959
0.9999899999999998
-4.664645983943594
0
164.03806521539855 0.9302879401594416
0.618397387944325
1.00001
-5.058749989249082
-1
164.03806521539855 0.9302879401594416
0.618397387944325
1.00001
-5.058749989249082
-2
166.73755445069008 0.9171250839684828
0.6085443594551707
1.0000099999999998
-5.443819250552041
-3
162.9037975319028 0.9235135639779989
0.6229724545672382
1.0
-5.547153936105078
-4
163.43828375957807 0.8997072047410588
0.6211227894667896
1.00001
-6.740016682070036
-5
163.41547394577023 0.9003297352831758
0.621676807217294
1.00002
-6.717739488824119
-6
163.01177448186428 0.8997934935707371
0.6220030740412733
1.0
-6.74314789014354
-7
162.95738405828774 0.8986089221296544
0.6212687480845583
0.9999900000000002
-6.809854418482575
-8
163.00640703614764 0.8998010930135755
0.6223468101479771
0.99999
-6.762887261410212
-9
163.00041490020863 0.8985435251157167
0.621154634387504
0.99999
-6.806451356106108
-10
165.44191838051506 0

In [93]:
df

,Discount,52W Discount,Weights
GHY,-11.00,-12.99,0.00186
FSD,-10.62,-11.21,0.12900
ISD,-10.31,-11.98,0.36150
EIC,-16.69,-23.92,0.00150
HYB,-12.02,-11.92,0.44650
AIF,-12.82,-14.04,0.05964
